### AI Judge to score compositions

This notebook uses the pre-trained model stored in this repository and the inference code to score compositions based on their similarity to Bach music. The overview of the notebook is below - 
1. Read input melodies from S3. Skip this step if you already have the input melodies that need to be scored in this local folder
2. Run inference on the input folder
3. Read results and sort them to find the winners!

_Note:
The inference code requires PyTorch, install it or use a pytorch kernel on SageMaker_

In [ ]:
%%capture
# install required packages
!pip install pypianoroll

In [ ]:
import os
import boto3
s3 = boto3.client('s3')

In [ ]:
bucket_name = '<bucket where .mid files are stored'

response = s3.list_objects_v2(Bucket=bucket_name)
# if storing the .mid files inside a prefix, mention the prefix like below-
# response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# download files locally
os.mkdir('input-folder')
for content in response['Contents']:
    s3.download_file(
        bucket_name,
        content['Key'],
        f"input-folder/{content['Key']}"
    )

In [ ]:
# run inference
os.mkdir('results')
!python inference.py --midi_folder input-folder/ --save_path results/

In [ ]:
# read results
import pickle
import pandas as pd

results = pickle.load(open('results/score_results.p', 'rb'))
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.reset_index()
results_df.columns=['Composer', 'Score']
results_df.sort_values(by='Score', ascending=False)

# Pick the first 'n' scores based on the number of winners for the challenge